<a href="https://colab.research.google.com/github/HisakaKoji/bert-japanese/blob/master/finetune_to_livedoor_corpus_20191221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning of the pretrained Japanese BERT model

Finetune the pretrained model to solve multi-class classification problems.  
This notebook requires the following objects:
- trained sentencepiece model (model and vocab files)
- pretraiend Japanese BERT model

Dataset is livedoor ニュースコーパス in https://www.rondhuit.com/download.html.  
We make test:dev:train = 2:2:6 datasets.

**This colab notebook assumes the above models are stored on some GSC bucket you can acess its objects.**

In [1]:
!git clone --recursive https://github.com/HisakaKoji/bert-japanese.git

Cloning into 'bert-japanese'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 274 (delta 7), reused 7 (delta 3), pack-reused 256
Receiving objects: 100% (274/274), 435.20 KiB | 5.51 MiB/s, done.
Resolving deltas: 100% (162/162), done.
Submodule 'bert' (https://github.com/google-research/bert.git) registered for path 'bert'
Cloning into '/content/bert-japanese/bert'...
remote: Enumerating objects: 336, done.        
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336        
Receiving objects: 100% (336/336), 291.40 KiB | 3.94 MiB/s, done.
Resolving deltas: 100% (184/184), done.
Submodule path 'bert': checked out '88a817c37f788702a363ff935fd173b6dc6ac0d6'


In [2]:
!pip install -q -r bert-japanese/requirements.txt

     |████████████████████████████████| 1.0MB 3.5MB/s 


In [3]:
from google.colab import auth
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
%cd bert-japanese/notebook

In [4]:
!gsutil cp gs://hisaka/model/wiki-ja.model ../model/
!gsutil cp gs://hisaka/model/wiki-ja.vocab ../model/

Copying gs://hisaka/model/wiki-ja.model...
Skipping attempt to download to filename ending with slash
(../model/). This typically happens when using gsutil to download from
a subdirectory created by the Cloud Console
(https://cloud.google.com/console)

Operation completed over 1 objects/786.8 KiB.                                    
Copying gs://hisaka/model/wiki-ja.vocab...
Skipping attempt to download to filename ending with slash
(../model/). This typically happens when using gsutil to download from
a subdirectory created by the Cloud Console
(https://cloud.google.com/console)

Operation completed over 1 objects/581.7 KiB.                                    


In [6]:
%ls

check-extract-features.ipynb       finetune-to-livedoor-corpus.ipynb
check-trained-tokenizer.ipynb      pretraining.ipynb
finetune_to_livedoor_corpus.ipynb


Check TPU.

In [7]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.10.32.186:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 8910512275326473485),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11773703224333706244),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6186601395256151883),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3303954787010643963),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17467741895312789961),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 6552579700939112602),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 8862163762629197284),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 12729527184096945373),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1119590821580

## Data preparation

You need to put preprocessed data on your GCS bucket.  
To create preprocessed data, follow https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb.

## Finetune pre-trained model

In [0]:
PRETRAINED_MODEL_PATH = 'gs://hisaka/model/model.ckpt-1400000'  # GCS bucket
PRETRAINED_MODEL_PATH = 'gs://hisaka/rurubu/output/model.ckpt-419'  # GCS bucket
INPUT_DATA_GCS = 'gs://hisaka/rurubu'  # GCS bucket
FINETUNE_OUTPUT_DIR = 'gs://hisaka/rurubu/output1219' # GCS bucket

In [9]:
%%time
!python3 ../src/run_classifier.py \
  --task_name=livedoor \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --do_train=true \
  --do_eval=true \
  --data_dir={INPUT_DATA_GCS} \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=64 \
  --learning_rate=2e-5 \
  --num_train_epochs=10.0 \
  --output_dir={FINETUNE_OUTPUT_DIR}




W1221 08:53:28.881185 139846685403008 module_wrapper.py:139] From ../src/run_classifier.py:661: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1221 08:53:28.881370 139846685403008 module_wrapper.py:139] From ../src/run_classifier.py:661: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1221 08:53:28.881757 139846685403008 module_wrapper.py:139] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1221 08:53:28.882394 139846685403008 module_wrapper.py:139] From ../src/run_classifier.py:682: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

Traceback (most recent call last):
  File "../src/run_classifier.py", line 856, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, ar

## Predict using the finetuned model

Let's predict test data using the finetuned model.  

In [10]:
%%time
!python3 ../src/run_classifier.py \
  --task_name=livedoor \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --do_train=false \
  --do_eval=false \
  --do_predict=true \
  --data_dir={INPUT_DATA_GCS} \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=64 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir={FINETUNE_OUTPUT_DIR}




W1221 08:53:34.064945 140251556906880 module_wrapper.py:139] From ../src/run_classifier.py:661: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1221 08:53:34.065144 140251556906880 module_wrapper.py:139] From ../src/run_classifier.py:661: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1221 08:53:34.065570 140251556906880 module_wrapper.py:139] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1221 08:53:34.066337 140251556906880 module_wrapper.py:139] From ../src/run_classifier.py:682: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

Traceback (most recent call last):
  File "../src/run_classifier.py", line 856, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, ar

## Evaluation

Download result and original data.

In [11]:
!gsutil cp {FINETUNE_OUTPUT_DIR}/test_results.tsv .
!gsutil cp {INPUT_DATA_GCS}/train.tsv .
!gsutil cp {INPUT_DATA_GCS}/dev.tsv .
!gsutil cp {INPUT_DATA_GCS}/test.tsv .

Copying gs://hisaka/rurubu/output1219/test_results.tsv...
/ [1 files][395.1 KiB/395.1 KiB]                                                
Operation completed over 1 objects/395.1 KiB.                                    
Copying gs://hisaka/rurubu/train.tsv...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      
Copying gs://hisaka/rurubu/dev.tsv...
/ [1 files][408.5 KiB/408.5 KiB]                                                
Operation completed over 1 objects/408.5 KiB.                                    
Copying gs://hisaka/rurubu/test.tsv...
/ [1 files][398.3 KiB/398.3 KiB]                                                
Operation completed over 1 objects/398.3 KiB.                                    


### Trained model

Check accuracy.

In [0]:
import numpy as np
import pandas as pd

In [13]:
import sys
sys.path.append("../src")

from run_classifier import LivedoorProcessor

processor = LivedoorProcessor()
label_list = processor.get_labels()

In [0]:
label_list = ['Traditional-Festivalsand-annual-events' , 'Traditional-Festivalsand-annual-events'  , 'Traditional-performing-arts-and-dance' ,'festival' ,'food'   ,'festival'   , \
          'flower-nature'   ,'festival'   ,'fireworks'   ,'snow'  ,'illumination'  ,'music'  ,'sports'  ,'museum'  ,'museum'  ,'festival'  ,'festival'  ,'experience'  ,  \
           'school'  ,'talk'  ,'stage'  ,'animal-fish-park'  ,'animal-fish-park'  ,'anniversary'  ,'fair'  ,'other'  ,'Industry'  ,'festival'  ,'festival' ,'other']

In [0]:
result = pd.read_csv("./test_results.tsv", sep='\t', header=None)

In [16]:
result.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.000069,0.000048,0.000019,0.000040,0.000061,0.000043,0.000111,0.000071,0.000082,0.000035,0.998055,0.000047,0.000078,0.000035,0.000027,0.000022,0.000048,0.000043,0.000039,0.000018,0.000020,0.000031,0.000033,0.000079,0.000030,0.000648,0.000038,0.000044,0.000046,0.000039
1,0.000158,0.000185,0.000323,0.000182,0.000277,0.000248,0.000154,0.000371,0.000296,0.989966,0.000140,0.000255,0.002022,0.000127,0.000119,0.000810,0.000337,0.000423,0.000139,0.000112,0.000301,0.000141,0.000195,0.000135,0.000124,0.001800,0.000154,0.000204,0.000147,0.000153


In [0]:
test_df = pd.read_csv("./test.tsv", sep='\t')

In [18]:
label_list

['Traditional-Festivalsand-annual-events',
 'Traditional-Festivalsand-annual-events',
 'Traditional-performing-arts-and-dance',
 'festival',
 'food',
 'festival',
 'flower-nature',
 'festival',
 'fireworks',
 'snow',
 'illumination',
 'music',
 'sports',
 'museum',
 'museum',
 'festival',
 'festival',
 'experience',
 'school',
 'talk',
 'stage',
 'animal-fish-park',
 'animal-fish-park',
 'anniversary',
 'fair',
 'other',
 'Industry',
 'festival',
 'festival',
 'other']

In [0]:
test_df['predict'] = [ label_list[np.array(elem[1]).argmax()] for elem in result.iterrows() ]

In [20]:
sum( test_df['label'] == test_df['predict'] ) / len(test_df)

0.8303393213572854

In [0]:
### 1/5 of full training data.
# sum( test_df['label'] == test_df['predict'] ) / len(test_df)

A littel more detailed check using `sklearn.metrics`.

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [23]:
print(classification_report(test_df['label'], test_df['predict']))

                                        precision    recall  f1-score   support

                              Industry       0.70      0.59      0.64        27
Traditional-Festivalsand-annual-events       0.88      0.88      0.88       320
 Traditional-performing-arts-and-dance       0.76      0.73      0.74        70
                           anniversary       0.00      0.00      0.00         1
                            experience       0.78      0.82      0.80        17
                                  fair       0.00      0.00      0.00         1
                              festival       0.58      0.62      0.60       104
                             fireworks       0.91      0.96      0.94       100
                         flower-nature       0.95      0.94      0.94       152
                                  food       0.77      0.83      0.80        24
                          illumination       0.90      0.95      0.92        92
                                museum 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
### 1/5 of full training data.
# print(classification_report(test_df['label'], test_df['predict']))

In [25]:
print(confusion_matrix(test_df['label'], test_df['predict']))

[[ 16   1   0   0   0   0   6   0   0   2   0   1   0   1   0   0   0   0]
 [  0 283  10   0   1   0  18   3   3   1   1   0   0   0   0   0   0   0]
 [  1  11  51   0   0   0   5   1   0   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0  14   0   0   0   0   0   0   0   2   0   0   1   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  2  15   4   1   0   0  64   4   5   2   2   0   3   1   0   1   0   0]
 [  0   1   0   0   0   0   2  96   0   0   1   0   0   0   0   0   0   0]
 [  0   3   1   0   0   0   2   0 143   0   1   0   0   0   0   2   0   0]
 [  2   0   0   0   0   0   2   0   0  20   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   0   0   0  87   0   0   3   1   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   2   0   0   0   0   0  18   0   0   0   0   0]
 [  0   4   1   0   0   0

In [0]:
### 1/5 of full training data.
# print(confusion_matrix(test_df['label'], test_df['predict']))

In [27]:
!ls

check-extract-features.ipynb	   pretraining.ipynb
check-trained-tokenizer.ipynb	   test_results.tsv
dev.tsv				   test.tsv
finetune_to_livedoor_corpus.ipynb  train.tsv
finetune-to-livedoor-corpus.ipynb


In [0]:
test_df.to_csv('../test20191221.csv')

In [29]:
!gsutil cp -r  test20191221.csv  gs://hisaka/20191220 

CommandException: No URLs matched: test20191221.csv


### Simple baseline model.

In [0]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
train_df = pd.read_csv("./train.tsv", sep='\t')
dev_df = pd.read_csv("./dev.tsv", sep='\t')
test_df = pd.read_csv("./test.tsv", sep='\t')

In [32]:
!apt-get -q install -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmecab2 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 7 not upgraded.
Need to get 12.8 MB of archives.
After this operation, 60.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-ipadic all 2.7.0-20070801+main-1 [12.1 MB]
Get:5 http://archive.ub

In [33]:
!pip install -q mecab-python3==0.7

     |████████████████████████████████| 51kB 2.0MB/s 


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import MeCab

In [0]:
m = MeCab.Tagger("-Owakati")

In [0]:
train_dev_df = pd.concat([train_df, dev_df])

In [0]:
train_dev_xs = train_dev_df['text'].apply(lambda x: m.parse(x))
train_dev_ys = train_dev_df['label']

test_xs = test_df['text'].apply(lambda x: m.parse(x))
test_ys = test_df['label']

In [0]:
vectorizer = TfidfVectorizer(max_features=750)
train_dev_xs_ = vectorizer.fit_transform(train_dev_xs)
test_xs_ = vectorizer.transform(test_xs)

The following set up is not exactly identical to that of BERT because inside Classifier it uses `train_test_split` with shuffle.  
In addition, parameters are not well tuned, however, we think it's enough to check the power of BERT.

In [0]:
%%time

model = GradientBoostingClassifier(n_estimators=200,
                                   validation_fraction=len(dev_df)/len(train_df),
                                   n_iter_no_change=5,
                                   tol=0.01,
                                   random_state=23)

### 1/5 of full training data.
# model = GradientBoostingClassifier(n_estimators=200,
#                                    validation_fraction=len(train_df)/len(dev_df),
#                                    n_iter_no_change=5,
#                                    tol=0.01,
#                                    random_state=23)

model.fit(train_dev_xs_, train_dev_ys)

In [0]:
print(classification_report(test_ys, model.predict(test_xs_)))

In [0]:
### 1/5 of full training data.
# print(classification_report(test_ys, model.predict(test_xs_)))

In [0]:
print(confusion_matrix(test_ys, model.predict(test_xs_)))

In [0]:
### 1/5 of full training data.
# print(confusion_matrix(test_ys, model.predict(test_xs_)))